## The above code cell is used for installing torch-geometric and dependencies

In [ ]:
# Add this in a Google Colab cell to install the correct version of Pytorch Geometric.
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

!pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-geometric 

Install project and data...

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

!cp -R ./drive/MyDrive/qa .
!cp -R ./drive/MyDrive/query2vec .

drive.flush_and_unmount()

In [ ]:
from query2vec.config import DEVICE

DEVICE

Training ...

In [ ]:
!python query2vec/main.py model --train_data='./qa/train_qa.txt' --val_data='./qa/val_qa.txt' \
                --epochs=10 

In [ ]:
!rm -rf model

Testing ...

In [ ]:
!python ./query2vec/test.py model/model.pth.tar hits@ --N=100 \
                  --test_data=./qa/test_qa.txt

In [ ]:
!python ./query2vec/test.py model/model.pth.tar mean_rank \
                  --test_data=./qa/test_qa.txt